# 서울 부동산 시장 분석 및 투자 전략 수립

## 과제 2: 아파트 규모별 가격 트렌드 및 생애주기 투자 전략

전용면적별(소형, 중형, 대형) 아파트의 가격 변동 추이를 분석, 생애주기와 데이터 분석에 따른 최적 투자 전략과 거주 전략을 제시

- 전용면적 기준으로 분류 후 규모별 가격 변동 추이 분석, 시각화
- 생애주기
  - 20~30: 신혼 / 첫 주택 구입 시기 
  - 40~50: 자산 증식 시기
  - 60~  : 다운사이징 / 현금화 단계

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from IPython.display import display

warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

df_sales = pd.read_csv('./data/sales_clean.csv')
df_sales['계약일'] = pd.to_datetime(df_sales['계약일'], format='%Y%m%d')

df_sales_copy = df_sales.copy()
df_sales_copy = df_sales_copy[df_sales_copy['공공임대'].isna()]
# display(df_sales_copy.head())

In [18]:
# 전용면적 기준 분류 및 필요 컬럼 추출
def area(df):
    if df['전용면적'] >= 85:
        return '대형'
    elif df['전용면적'] >= 60:
        return '중형'
    else:
        return '소형'

df_area_analysis = pd.DataFrame({
    '전용면적': df_sales_copy['전용면적'],
    '계약월': pd.to_datetime(df_sales_copy['계약월'], format='%Y%m'),
    '건축년도': df_sales_copy['건축년도'],
    '거래금액': df_sales_copy['거래금액'],
    '평단가': df_sales_copy['평단가'],
    '면적구분': df_sales_copy.apply(area, axis=1) 
})

display(df_area_analysis.head())

,전용면적,계약월,건축년도,거래금액,평단가,면적구분
0,84.980,2020-12-01,2013,820000000,31894205,중형
1,240.305,2020-12-01,2011,7100000000,97675058,대형
2,84.880,2020-12-01,2018,1800000000,70093458,중형
3,104.220,2020-12-01,1978,595000000,18870917,대형
4,84.920,2020-12-01,2001,1350000000,52549630,중형


In [ ]:
# 핵심 지표 추출
## 1. 월별 평균 거래가, 평단가
monthly_stats = df_area_analysis.groupby(['면적구분', '계약월']).agg({
    '거래금액': ['count', 'mean'],
    '평단가': 'mean'
}).reset_index()
monthly_stats.columns = ['면적구분', '계약월', '월별거래건수', '월평균거래금액', '월평균평단가']

## 2. 월별 수익률 계산

,면적구분,계약월,월별거래건수,월평균거래금액,월평균평단가
0,대형,2020-01-01,897,1.052286e+09,2.811883e+07
1,대형,2020-02-01,1176,1.082832e+09,2.921661e+07
2,대형,2020-03-01,623,1.133290e+09,3.012994e+07
3,대형,2020-04-01,502,1.306891e+09,3.362868e+07
4,대형,2020-05-01,1038,1.364007e+09,3.524010e+07
...,...,...,...,...,...
193,중형,2025-02-01,2662,1.535431e+09,6.154794e+07
194,중형,2025-03-01,4133,1.456529e+09,5.846085e+07
195,중형,2025-04-01,2152,1.166401e+09,4.734206e+07
196,중형,2025-05-01,3060,1.304678e+09,5.239205e+07


In [ ]:


## 2. 변동성
### 월별 데이터 생성
monthly_stats = df_popular.groupby(['구', '계약월']).agg({'거래금액': ['mean', 'count']}).reset_index()
monthly_stats.columns = ['구', '계약월', '월평균거래금액', '월별거래건수']
monthly_stats['계약월'] = pd.to_datetime(monthly_stats['계약월'], format='%Y%m')
monthly_stats = monthly_stats.sort_values(['구', '계약월'])

### 월별 수익률 확인
monthly_return = monthly_stats[['구','계약월','월평균거래금액']]
monthly_return['월별수익률'] = monthly_return.groupby('구')['월평균거래금액'].transform(lambda x: np.log(x / x.shift(1)))
# display(monthly_return.head())


### 연수익률 확인
monthly_return['연도'] = monthly_return['계약월'].dt.year
annual_return = monthly_return.groupby(['구','연도'])['월별수익률'].apply(lambda x:np.prod(1 + x) - 1).reset_index()
annual_return.rename(columns={'월별수익률': '연수익률'}, inplace=True)
# display(annual_return.head())

### 변동성(월수익률의 표준편차)
monthly_return['연도'] = monthly_return['계약월'].dt.year
volatility = monthly_return.groupby(['구', '연도'])['월별수익률'].std().reset_index()
volatility.rename(columns={'월별수익률': '연도별변동성'}, inplace=True)
# display(volatility.head())

## 3. 거래밀도 (월별 거래건수 / 인구수)

### 구별/월별 인구수 데이터
population_df = pd.read_csv('./data/등록인구(월별).csv', header=[0, 1])
population_df = population_df.drop(columns=[population_df.columns[0]])
population_df.columns = ['_'.join(filter(None, map(str, col))).strip() for col in population_df.columns.values]

population_df_long = population_df.melt(id_vars=['동별(2)_동별(2)'], var_name='계약월_항목', value_name='값')
population_df_long[['계약월','항목']] = population_df_long['계약월_항목'].str.split('_', expand=True)
population_df_long['계약월'] = pd.to_datetime(population_df_long['계약월'].str.replace(" ", "").str.replace(".", "-"), format='%Y-%m')

### Pivot
df_population = population_df_long.pivot_table(
    index=['동별(2)_동별(2)', '계약월'],
    columns='항목',
    values='값'
).reset_index()
df_population.rename(columns={
    '동별(2)_동별(2)': '구',
    '세대 (세대)': '세대수',
    '합계 (명)': '총인구'
}, inplace=True)

population_popular_mask = df_population['구'].isin(popular + unpopular)
population_density = pd.merge(df_population[population_popular_mask], monthly_stats[['구','계약월','월별거래건수']], on=['구', '계약월'], how='left')

### 거래밀도 계산(세대수, 총인구별)
population_density['거래밀도_총인구'] = population_density['월별거래건수'] / population_density['총인구']
population_density['거래밀도_세대수'] = population_density['월별거래건수'] / population_density['세대수']
# display(population_density)

## 12개월 이동평균, 이동표준편차
monthly_stats = monthly_stats.sort_values(['구', '계약월'])

# 이동평균과 이동표준편차 계산 (월평균거래금액 기준)
monthly_stats['이동평균'] = monthly_stats.groupby('구')['월평균거래금액'].transform(lambda x: x.rolling(12).mean())
monthly_stats['이동표준편차'] = monthly_stats.groupby('구')['월평균거래금액'].transform(lambda x: x.rolling(12).std())
# display(monthly_stats.head(20))